<a href="https://colab.research.google.com/github/ducvu/ml_projects/blob/master/scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install selenium
!apt-get update  # Update package lists
!apt install -y firefox-geckodriver # Install GeckoDriver for Firefox, use ChromeDriver if you prefer Chrome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.1 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [963 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https:

In [3]:
# Import necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
from io import StringIO
import urllib.parse

# Set up Firefox driver for Selenium
options = webdriver.FirefoxOptions()
options.add_argument('--headless')  # Run in headless mode for faster execution without GUI
driver = webdriver.Firefox(options=options)

# Define the base URL and parameters
base_url = "https://www.cvedetails.com/vulnerability-search.php?"
params = {
    "f": "1",
    "vendor": "",
    "product": "",
    "cweid": "",
    "cvssscoremin": "",
    "cvssscoremax": "",
    "publishdatestart": "2020-01-01",  # Adjust the start date as needed
    "publishdateend": "2020-12-31",    # Adjust the end date as needed
    "updatedatestart": "",
    "updatedateend": "",
    "cisaaddstart": "",
    "cisaaddend": "",
    "cisaduestart": "",
    "cisadueend": "",
    "optexeccode": "1",
    "page": "1"  # Start from the first page
}

# Initialize DataFrame to store results
columns = ["CVE", "Published", "Last.Update", "Max.CVSS.Base.Score", "EPSS.Score",
           "CISA.KEV.Added", "Public.Exploit.Exists", "Summary", "Year"]
df_all = pd.DataFrame(columns=columns)

# Function to construct URL
def construct_url(params):
    return base_url + urllib.parse.urlencode(params)

# Start scraping loop
page = 1
while (page < 5):
    print(f"Scraping page {page}...")

    # Update the page parameter
    params['page'] = str(page)
    current_url = construct_url(params)

    # Navigate to the current URL
    driver.get(current_url)
    time.sleep(4)  # Wait for the page to load

    try:
        # Click to copy the output data
        copy_button = driver.find_element(By.XPATH, '//a[@href = "javascript:copyResultsAsTSV()"]')
        copy_button.click()
        time.sleep(2)  # Wait for the copy action to complete

        # Extract data from clipboard
        clipboard_content = driver.execute_script("return navigator.clipboard.readText();")

        if clipboard_content.strip() == "":
            print("No more data available or no data on this page.")
            break

        # Read data into a DataFrame
        df = pd.read_csv(StringIO(clipboard_content), sep='\t')

        # Add Year column for the current data
        df['Year'] = params['publishdatestart'][:4]
        df_all = pd.concat([df_all, df], ignore_index=True)

        # Increment page number
        page += 1

    except Exception as e:
        print("No more pages or error occurred:", e)
        break  # Exit loop if no data or an error occurs

# Close the browser
driver.quit()

# Display the result
print(df_all.head())

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
              CVE   Published Last.Update Max.CVSS.Base.Score EPSS.Score  \
0  CVE-2020-35863  2020-12-31         NaN                 NaN        NaN   
1  CVE-2020-35858  2020-12-31         NaN                 NaN        NaN   
2  CVE-2020-35657  2020-12-23         NaN                 NaN        NaN   
3  CVE-2020-35656  2020-12-23         NaN                 NaN        NaN   
4  CVE-2020-35627  2020-12-28         NaN                 NaN        NaN   

  CISA.KEV.Added Public.Exploit.Exists  \
0            NaN                   NaN   
1            NaN                   NaN   
2            NaN                   NaN   
3            NaN                   NaN   
4            NaN                   NaN   

                                             Summary  Year Last Update  \
0  An issue was discovered in the hyper crate bef...  2020  2021-07-21   
1  An issue was discovered in the prost crate bef...  2020  2021-0